# 大语言模型Transformer库-预训练流程编码体验

## 下载模型和数据集
```bash
git clone https://huggingface.co/hfl/rbt3
git clone https://huggingface.co/datasets/dirtycomputer/ChnSentiCorp_htl_all
```

## 步骤1：导入相关依赖

In [ ]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification

model_name_or_path = "/root/pretrains/rbt3"
dataset_path = "/root/pretrains/ChnSentiCorp_htl_all/ChnSentiCorp_htl_all.csv"

## 步骤2：获取数据集

In [ ]:
import pandas as pd
data = pd.read_csv(dataset_path)
data.dropna()
data

## 步骤3：构建数据集

In [ ]:
from torch.utils.data import Dataset

import pandas as pd

class MyDataset(Dataset):
    def __init__(self) -> None:
        super().__init__()
        self.data = pd.read_csv(dataset_path)
        self.data = self.data.dropna()
    def __getitem__(self,index):
        return self.data.iloc[index]["review"], self.data.iloc[index]["label"]
    def __len__(self):
        return len(self.data) 

dataset = MyDataset()
for i in range(5):
    print(dataset[i])

## 步骤4：划分数据集

In [ ]:
from torch.utils.data import  random_split

trainset, validset = random_split(dataset,lengths=[0.9,0.1])
len(trainset),len(validset)

## 步骤5：创建DataLoader

In [ ]:
import torch
from torch.utils.data import DataLoader

tokenizer = AutoTokenizer.from_pretrained(model_name_or_path)

def collate_func(batch):
    texts,labels=[],[]
    for item in batch:
        texts.append(item[0])
        labels.append(item[1])
        ## return_tensors="pt" 返回的是pytorch tensor类型。
        ## 吃葡萄不吐葡萄皮
        ## 不吃葡萄到吐葡萄皮
    inputs = tokenizer(texts,max_length=128,padding="max_length",truncation=True, return_tensors="pt")
    inputs["labels"] = torch.tensor(labels)
    return inputs
## dataloader中设置shuffle值为True，表示每次加载的数据都是随机的，将输入数据的顺序打乱。shuffle值为False，
## 表示输入数据顺序固定。

trainloader = DataLoader(trainset,batch_size=32,shuffle=True,collate_fn=collate_func)
validloader = DataLoader(validset,batch_size=64,shuffle=False,collate_fn=collate_func)

next(enumerate(validloader))[1]

## 步骤6：创建模型及其优化器

In [ ]:
from torch.optim import Adam

from transformers import AutoModelForSequenceClassification

model = AutoModelForSequenceClassification.from_pretrained("/root/pretrains/rbt3")

if torch.cuda.is_available():
    model = model.cuda()
"""
当我们训练一个机器学习模型时，我们需要选择一个优化算法来帮助我们找到模型参数的最佳值。这个优化算法就是优化器（optimizer）。

在这行代码中，我们选择了一种叫做Adam的优化算法作为我们的优化器。Adam算法是一种常用的优化算法，
它根据每个参数的梯度（即参数的变化率）和学习率（lr）来更新参数的值。

"model.parameters()"表示我们要优化的是模型的参数。模型的参数是模型中需要学习的权重和偏置等变量。

"lr=2e-5"表示学习率的值被设置为2e-5（即0.00002）。学习率是控制模型在每次迭代中更新参数的步长。较大的学习率可能导致模型无法收敛，
而较小的学习率可能需要更长的训练时间
"""
optimizer = Adam(model.parameters(), lr=2e-5)

## 步骤7：训练与验证

In [ ]:
# 定义一个训练和评估的函数
# 设定训练循环，包括前向传播、计算损失、反向传播和权重更新。同时，定期在验证集上检查模型性能，以监控过拟合情况并及时停止训练。
def evaluate():
    ## 将模型设置为评估模式
    model.eval()
    acc_num=0
    #将训练模型转化为推理模型，模型将使用转换后的推理模式进行评估
    with torch.inference_mode():
        for batch in validloader:
            ## 检查是否有可用的GPU，如果有，则将数据批次转移到GPU上进行加速
            if torch.cuda.is_available():
                batch = {k: v.cuda() for k,v in batch.items()}
            ##对数据批次进行前向传播，得到模型的输出
            output = model(**batch)
            ## 对模型输出进行预测，通过torch.argmax选择概率最高的类别。
            pred = torch.argmax(output.logits,dim=-1)
            ## 计算正确预测的数量，将预测值与标签进行比较，并使用.float()将比较结果转换为浮点数，使用.sum()进行求和操作
            acc_num += (pred.long() == batch["labels"].long()).float().sum()
    ## 返回正确预测数量与验证集样本数量的比值，这表示模型在验证集上的准确率
    return acc_num / len(validset)

def train(epoch=3,log_sep=100):
    global_step = 0
    for ep in range(epoch):
        ## 开启训练模式
        model.train()
        for batch in trainloader:
            if torch.cuda.is_available():
                batch = {k: v.cuda() for k, v in batch.items()}
            ## 梯度归0
            optimizer.zero_grad()
            ## 对数据批次进行前向传播，得到模型的输出
            output=model(**batch)
            ## 计算损失函数梯度并进行反向传播
            output.loss.backward()
            ## 优化器更新
            optimizer.step()
            if(global_step % log_sep == 0):
                print(f"epoch:{ep},step:{global_step},loss:{output.loss.item()}")
            global_step += 1
        ## 准确率
        acc = evaluate()
        ## 第几轮
        print(f"epoch:{ep},acc:{acc}")

# 训练
train()

## 步骤8：模型预测

In [ ]:
#sen = "我昨晚在酒店里睡得非常好"
sen ="昨天我在酒店睡觉发现被子有一股霉味"

id2label = {0:"差评",1:"好评"}
## 将模型设置为评估模式
model.eval

 #将训练模型转化为推理模型，模型将使用转换后的推理模式进行评估
with torch.inference_mode():
    ## 分词&&向量化
    inputs = tokenizer(sen,return_tensors = "pt")
    ## GPU加速
    inputs = {k:v.cuda() for k,v in inputs.items()}
    ## 进行预测
    logits=model(**inputs).logits
    ## 在logits的最后一个维度上找到最大值，并返回其所在的索引。这相当于选择模型认为最有可能的类别
    pred = torch.argmax(logits, dim = -1)
    
    print(f"输入：{sen} \n模型的预测结果：{id2label.get(pred.item())}")